In [1]:
import torch
from config import BitformerConfig
from model_zoo import VisionBitformerForImageClassification, VisionBitformerForSemanticSegmentation
from metrics import compute_metrics_single_label_classification
from trainer import get_trainer
from data_zoo import get_vision_dataset, vision_collator
from utils import get_yaml

In [2]:
yaml_path = './yamls/vision/small.yaml'
args = get_yaml(yaml_path)

In [3]:
cfg = BitformerConfig(**args['model_config'], num_labels=args['general_config']['num_labels'])
model = VisionBitformerForImageClassification(config=cfg)
model.num_parameters() / 1e6

1.131476

In [4]:
model

VisionBitformerForImageClassification(
  (model): VisionBitformer(
    (embed_tokens): Embedding(32000, 28)
    (layers): ModuleList(
      (0): BitformerLayer(
        (self_attn): VisionAttention(
          (qkv): BitLinear(
            in_features=28, out_features=1536, bias=False
            (norm): LayerNorm((28,), eps=1e-05, elementwise_affine=True)
          )
          (o): BitLinear(
            in_features=512, out_features=28, bias=False
            (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (rotary_emb): RotaryEmbedding()
        )
        (MLP): VisionMoeBlock(
          (router): Linear(in_features=28, out_features=4, bias=False)
          (experts): ModuleList(
            (0-3): 4 x MLP(
              (w1): BitLinear(
                in_features=28, out_features=512, bias=False
                (norm): LayerNorm((28,), eps=1e-05, elementwise_affine=True)
              )
              (w2): BitLinear(
                in_features=51

In [6]:
test = torch.rand(8, 28, 28)
labels = torch.randint(0, 9, (1, ))
out = model(test, labels=labels)

RuntimeError: shape '[8, 8, 28, -1]' is invalid for input of size 6272

In [6]:
out.logits.shape

torch.Size([8, 10, 28, 28])

In [9]:
data_path = 'mnist'
train_dataset, test_dataset = get_vision_dataset(data_path)

In [15]:
trainer = get_trainer(model=model,
                      train_dataset=train_dataset,
                      valid_dataset=test_dataset,
                      compute_metrics=compute_metrics_single_label_classification,
                      data_collator=vision_collator,
                      **args['training_args'])

In [16]:
trainer.train()
trainer.evaluate()

  0%|          | 0/9380 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1792x1792 and 28x4)